In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from skimage.util import view_as_windows

# 从 cwSaab.py 导入 cwSaab 类
from cwSaab import cwSaab

# 定义一个函数来获取数据的窗口视图
def Shrink(X, shrinkArg):
    win = shrinkArg['win']
    stride = shrinkArg['stride']
    ch = X.shape[-1]
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    return X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)

# 加载 MNIST 数据
digits = datasets.load_digits()
X = digits.images.reshape((len(digits.images), 8, 8, 1))  # 将数据调整为 (N, H, W, C) 格式
y = digits.target

# 分割数据为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 设置 Saab 参数
SaabArgs = [{'num_AC_kernels':-1, 'needBias':True, 'cw':False}, 
            {'num_AC_kernels':5, 'needBias':True, 'cw':True}]
shrinkArgs = [{'func': Shrink, 'win':5, 'stride': 1}, 
              {'func': Shrink, 'win':5, 'stride': 1}]

# 初始化 cwSaab 类
cwsaab = cwSaab(depth=2, TH1=0.005, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs)

# 训练模型
cwsaab.fit(X_train)

# 变换训练数据和测试数据
train_features = cwsaab.transform(X_train)
test_features = cwsaab.transform(X_test)

# 使用 XGBoost 进行分类
from xgboost import XGBClassifier
classifier = XGBClassifier(n_estimators=100)
classifier.fit(np.concatenate(train_features, axis=-1).reshape(len(X_train), -1), y_train)

# 测试准确率
test_accuracy = classifier.score(np.concatenate(test_features, axis=-1).reshape(len(X_test), -1), y_test)
print(f"Test Accuracy: {test_accuracy:.2%}")
